# Idea
The idea is that we predict rotated linear dimension position from lines and text from the drawing.

# Thoughts
Basic logic is we split dataset to input lines or texts positions StartPoint, EndPoint, Position XYZ and predict dimension extension line poistion XLine1Point, XLine2Point XYZ.

We going to group samples by FileId. That is each sample will contain variable length data (attributes of variable number of  lines and text) and variable output data (variable number of dimensions).

I intend to [use RNN for it](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)

In [2]:
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
from processing import *

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
test_data = pd.read_pickle('test_dataset.pickle')
# BUG: Dataset is filtered, we need all data from SOME files, not SOME data from All Files
test_data

,ClassName,FileId,XLine1Point.X,XLine1Point.Y,XLine1Point.Z,XLine2Point.X,XLine2Point.Y,XLine2Point.Z,StartPoint.X,StartPoint.Y,StartPoint.Z,EndPoint.X,EndPoint.Y,EndPoint.Z,Position.X,Position.Y,Position.Z
867872,AcDbText,0d9e42c4-bded-4dd0-9bbb-d4abad30410e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,621037.257604,12177.425365,-17504.791854
310838,AcDbLine,6f4a5e84-b45f-4d1f-9d80-a3a0d159c145,NaN,NaN,NaN,NaN,NaN,NaN,2.103418e+02,340.502524,0.0,2.103418e+02,350.418200,0.0,NaN,NaN,NaN
447789,AcDbLine,50f2c8eb-f9f2-4470-a9d0-2df65db33347,NaN,NaN,NaN,NaN,NaN,NaN,1.584164e+06,664570.788057,0.0,1.584164e+06,664545.788289,0.0,NaN,NaN,NaN
222199,AcDbLine,33cac39f-f024-40d4-98c9-d94554cfe7cf,NaN,NaN,NaN,NaN,NaN,NaN,5.793491e+03,461.181698,0.0,5.793491e+03,461.179070,0.0,NaN,NaN,NaN
388506,AcDbLine,7c0e80ee-c625-4601-bf74-4f689aace0b3,NaN,NaN,NaN,NaN,NaN,NaN,7.404816e+02,472.590296,0.0,7.404816e+02,472.590296,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688070,AcDbLine,6a359cb6-c992-44da-87e2-a277a928a525,NaN,NaN,NaN,NaN,NaN,NaN,-1.055062e+05,-87863.162449,10800.0,-1.056038e+05,-87895.604995,10800.0,NaN,NaN,NaN
726705,AcDbLine,1c68b718-fb88-4678-9c2e-512776ea79dd,NaN,NaN,NaN,NaN,NaN,NaN,1.829456e+06,-703357.579645,0.0,1.829456e+06,-701517.579645,0.0,NaN,NaN,NaN
432628,AcDbLine,7cdbf03f-a841-41a9-8279-5fd0da031a6d,NaN,NaN,NaN,NaN,NaN,NaN,1.516576e+03,636.108325,0.0,1.515976e+03,636.108325,0.0,NaN,NaN,NaN
458942,AcDbLine,5b508844-6c5a-41de-a401-899f3c9ba6f9,NaN,NaN,NaN,NaN,NaN,NaN,2.570462e+03,1481.669321,0.0,2.570462e+03,1482.329321,0.0,NaN,NaN,NaN


# Dataset implementation

In [103]:
# https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
import torch
from torch import nn
from torch.utils.data import Dataset, SubsetRandomSampler
from torch.nn import functional as F

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
device

device(type='cpu')

In [249]:
x_columns = [
    #'ClassName', 
    'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',
    'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z',
    'Position.X', 'Position.Y', 'Position.Z']

y_columns = [
    #'ClassName', 
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z']

ent_features = len(x_columns)
dim_features = len(y_columns)

In [250]:
class EntityDataset(Dataset):
    def __init__(self, pandasData):
        self.data = pandasData
        self.groupped = pandasData.groupby('FileId')
        self.keys = list(self.groupped.groups.keys() )
        
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, index):
        x = self.groupped.get_group(self.keys[index])[x_columns]
        y = self.groupped.get_group(self.keys[index])[y_columns]
        
        return torch.tensor(np.array(x.values)), torch.tensor(np.array(y.values))

In [251]:
entities = EntityDataset(test_data)
c,d = entities[3]
print(c.shape, d.shape)

torch.Size([10, 9]) torch.Size([10, 6])


In [252]:
batch_size = 2

data_len = len(entities)

validation_fraction = 0.2
test_fraction       = 0.3

val_split  = int(np.floor(validation_fraction * data_len))
test_split = int(np.floor(test_fraction * data_len))
indices = list(range(data_len))
np.random.seed(228)

np.random.shuffle(indices)

val_indices   = indices[:val_split]
test_indices  = indices[val_split:test_split]
train_indices = indices[test_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler   = SubsetRandomSampler(val_indices)
test_sampler  = SubsetRandomSampler(test_indices)

# https://stackoverflow.com/questions/64586575/adding-class-objects-to-pytorch-dataloader-batch-must-contain-tensors
def custom_collate(sample):
    x = []
    y = []

    for (xx, yy) in sample:
        x.append(xx)
        y.append(yy)
    #print(len(x), len(y))
    return x, y

train_loader = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = train_sampler, collate_fn=custom_collate)
val_loader   = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = val_sampler)
test_loader  = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = test_sampler)

In [253]:
(a,b) = next(iter(train_loader))
print(len(a),len(b))
for (x,y) in iter(train_loader):
    for xx in x:
        print(xx.shape)
        
        pass
    # print(a.shape,b.shape)
    break

2 2
torch.Size([125, 9])
torch.Size([21, 9])


# Model and training


In [341]:
class Encoder(nn.Module):
    def __init__(self, ent_features, learned_size, mlp_h = 64):
        super(Encoder, self).__init__()
        
        # mlp_h : multilayer perceptron hidden size
        self.mlp_h = mlp_h
        self.learned_size = learned_size
        
        self.conv1 = nn.Conv1d(ent_features, mlp_h, 1)
        self.bn1 = nn.BatchNorm1d(mlp_h)
        self.output = nn.Linear(mlp_h, learned_size)
        
    def forward(self, x):
        batch_len = len(x)
        sample = x[0]
        
        num_ent = sample.shape[0]
        ent_features = sample.shape[1]
        
        result = torch.Tensor(batch_len, self.learned_size)
        i = 0
        
        for xx in x:
            xx = xx.to(device)
            print(xx.shape)
            
            # batch x channels x features
            v = torch.Tensor(xx.shape[0], 1, xx.shape[1])
            v[:, 0, :] = xx[:, :]
            print(v.shape)
            v = v.transpose(1,2)
            v = self.conv1(v)
            v = self.bn1(v)
            v = F.relu(v)
            print(v.shape)
            v = torch.max(v,2).values
            print(v.shape)
            v = self.output(v)
            print(v.shape)
            
            result[i,:] = torch.max(v,0).values
            i = i + 1
            
        return result
    
class Decoder(nn.Module):
    def __init__(self, learned_size, dim_features, mlp_h = 64):
        super(Decoder, self).__init__()
        
        self.dim_features = dim_features
        self.learned_size = learned_size
        self.mlp_h = mlp_h
        
        self.mlp = nn.Sequential(
            nn.Linear(learned_size, mlp_h),
            torch.nn.BatchNorm1d(mlp_h),
            nn.ReLU(inplace = True),
            nn.Linear(mlp_h, dim_features)
        )
        
    def forward(self, x, num_dims):
        batch_size = x.shape[0]
        
        x = self.mlp(x)
        

In [342]:
# learned size 
ls = 128

encoder = Encoder(9, ls)
decoder = Decoder(ls, 6)

for idx, (inp, outp) in enumerate(train_loader):
    for inpu in inp:
        print("input:", inpu.shape)
    learned = encoder(inp)
    print("learned shape:", learned.shape)
    
    decoded = decoder(learned, 3)
    print("decoded shape:", decoded.shape)
    
    for outpu in outp: 
        print("output:", outpu.shape)
    break

input: torch.Size([26, 9])
input: torch.Size([6, 9])
torch.Size([26, 9])
torch.Size([26, 1, 9])
torch.Size([26, 64, 1])
torch.Size([26, 64])
torch.Size([26, 128])
torch.Size([6, 9])
torch.Size([6, 1, 9])
torch.Size([6, 64, 1])
torch.Size([6, 64])
torch.Size([6, 128])
learned shape: torch.Size([2, 128])


AttributeError: 'NoneType' object has no attribute 'shape'

In [339]:
class RnnEncoder(nn.Module):
    def __init__(self, ent_features, learned_size, n_neurons = 16):
        super(RnnEncoder, self).__init__()

        self.rnn = nn.RNN(input_size = ent_features, hidden_size = learned_size)
        self.learned_size = learned_size
        self.ent_features = ent_features
        
        self.fcn = nn.Sequential(
            nn.Linear(learned_size, 1),
            nn.ReLU()
        )
        
    def init_hidden(self,batch_size):
        self.hidden = torch.zeros(1, batch_size, self.learned_size)
        
    def forward(self, x):
        # input of shape (seq_len, batch, input_size)
        batch_size = len(x)
        
        hiddens = torch.zeros([batch_size, self.learned_size], dtype = torch.float32, device = device)
        outs = torch.zeros([batch_size, 1], dtype = torch.int32, device = device)
        for j in range(batch_size):
            f = x[j]
            entities_count = f.shape[0]
            self.init_hidden(entities_count)
            inp = torch.zeros([entities_count, 1, self.ent_features], dtype = torch.float32, device = device)
            # print(inp.shape, f.shape)
            inp[:,0,:] = f[:,:]
            
            outp, h_n = self.rnn(inp)
            # print('h_N', h_n.shape)
            # print('cell out', outp.shape)
            n = self.fcn(outp)
            
            # https://stackoverflow.com/questions/48005152/extract-the-count-of-positive-and-negative-values-from-an-array
            cou = np.sum(np.array(n.detach().numpy()) > 0, axis = 0)
            # print(cou)
            outs[j] = torch.tensor(cou) 
            hiddens[j] = h_n
            j = j + 1
                
        return outs, hiddens
            
class RnnDecoder(nn.Module):
    def __init__(self, learned_size, dim_features):
        super(RnnDecoder, self).__init__()
        self.rnn = nn.RNN(input_size = learned_size, hidden_size = dim_features)
        self.learned_size = learned_size
        self.dim_features = dim_features
        
    def forward(self, x, out_counts):
        '''Receives tensor with hidden state and number of dimensions tensor per each file.
            Returns a list of tensors with dimension parameters'''
        # input of shape (seq_len, batch, input_size)
        
        result = []
        for j in range(batch_size):
            dim_count = out_counts[j]
            outs = 0
            if dim_count > 0:
                inp = torch.zeros([dim_count, 1, self.learned_size], dtype=torch.float32, device = device)
                xj = x[j]
                print(inp.shape, xj.shape)
                inp[:,0] = xj
                outs, hiddens = self.rnn(inp)
            result.append(outs)
        
        return result

In [340]:
rnn_encoder = RnnEncoder(ent_features, 1024)
rnn_decoder = RnnDecoder(1024, dim_features)
(x,y) = next(iter(train_loader))
outs, learned = rnn_encoder(x)
print('learned shape:', learned.shape, "outs shape:", outs.shape, outs)
# make sure something is passed
outs[0] = 2
decoded = rnn_decoder(learned, outs)
print('decoded:', decoded)

learned shape: torch.Size([2, 1024]) outs shape: torch.Size([2, 1]) tensor([[0],
        [0]], dtype=torch.int32)
torch.Size([2, 1, 1024]) torch.Size([1024])
decoded: [tensor([[[nan, nan, nan, nan, nan, nan]],

        [[nan, nan, nan, nan, nan, nan]]], grad_fn=<StackBackward>), 0]


/home/mk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


In [240]:
# https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
# shape coordination
# I need output like [files_number x dimensions_per_file x dimension_features]
# and input like [files_number x entities_per_file x entities_features]
def test_rnn_shapes():
    in_size_features = 9
    out_size_features = 6
    hidden_size = out_size_features
    num_dimensions = 66
    batch_l = 1
    num_entities = 166
    num_rnn_layers = 7
    rnn = torch.nn.RNN(input_size = in_size_features, hidden_size = hidden_size, num_layers = num_rnn_layers, batch_first = True)
    inpt = torch.randn(batch_l, num_entities, in_size_features)
    h0 = torch.randn( num_rnn_layers, batch_l,  hidden_size)
    print ('input.shape', inpt.shape, 'h0.shape', h0.shape)
    out, hn= rnn(inpt, h0)
    print ('out.shape', out.shape, 'hn.shape', hn.shape)
    
    # flatten shape of output
    linin = torch.flatten(out, start_dim = 1)
    print("flattened output of rnn", linin.shape)
    lin = torch.nn.Linear(in_features = num_entities*out_size_features, out_features = num_dimensions*out_size_features)
    s = lin(linin)
    
    # unflatten shape of output
    s = s.view(batch_l,num_dimensions, out_size_features)
    print("output shape", s.shape)
    
    
test_rnn_shapes()

input.shape torch.Size([1, 166, 9]) h0.shape torch.Size([7, 1, 6])
out.shape torch.Size([1, 166, 6]) hn.shape torch.Size([7, 1, 6])
flattened output of rnn torch.Size([1, 996])
output shape torch.Size([1, 66, 6])


In [12]:
def plot_history(loss_history, train_history, val_history):
    plt.ylabel('Accuracy @ epoch')
    
    train, = plt.plot(train_history)
    train.set_label("train")
    validation, = plt.plot(val_history)
    validation.set_label("validation")

    plt.legend()
    plt.show()

In [13]:
def compute_accuracy(model, loader):
    pass

def train_model(model, train_loader, val_loader, loss, optimizer, epochs):
    start = time.time()
    
    loss_history = []
    train_istory = []
    val_history  = []
    
    for epoch in range(epochs):
        model.train()
        
        loss_accumulated = 0
        
        i_step = 1
        for (x,y) in enumerate(train_loader):
            optimizer.zero_grad()
            
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            output, hidden = model(x_gpu)
            loss_value = loss(output, y_gpu)
            
            loss_value.backward()
            optimizer.step()
            
            loss_accumulated += loss_value
            
            #val_accuracy = compute_accuracy(model, val_loader)
            print('[{0} @ {4} sec] Loss: {1:4f} Train err: {2:2.2f}% Val err: {3:2.2f}%'. format(
                epoch,
                ave_loss,
                (1 - train_accuracy) * 100,
                (1 - val_accuracy) * 100,
                time.time() - start
            ))
        
        # loss_history.append(float(loss_accum / i_step))
        # val_history.append(val_accuracy)
        # clear_output(wait=True)
        
        i_step += 1
        
    plot_history(loss_history, train_history, val_history)

In [14]:
lr = 1e-3
epochs = 3
optimizer = optim.Adam(lr = lr)
loss = nn.MSELoss()
model = RNN()

model.to(device)
train_model(model,train_loader, val_loader, loss, optimizer, epochs)

NameError: name 'optim' is not defined

In [15]:
torch.cuda.empty_cache()
pre
model.eval()
for (x,y) in test_loader:
    with torch.no_grad():
        output = model.forward(x.to(device))
        accuracy = compute_accuracy(model, test_loader)

NameError: name 'pre' is not defined